In [ ]:
import numpy as np
from numpy.random import seed
seed(1)
from zipfile import ZipFile

import pandas as pd
from sklearn.model_selection import train_test_split
from skimage.io import imread
from tqdm import tqdm

import torch
from torch import Tensor, unsqueeze, optim, device
import torch.nn as nn
import torch.nn.functional as F
from torch import cat

if torch.cuda.is_available():
    print("using GPU")
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
zip_path = "../input/tgs-salt-identification-challenge/train.zip"
with ZipFile(zip_path,'r') as z:
    z.extractall("./train")

zip_path = "../input/tgs-salt-identification-challenge/test.zip"
with ZipFile(zip_path,'r') as z:
    z.extractall('./test')

In [ ]:
train_df = pd.read_csv("../input/tgs-salt-identification-challenge/train.csv", index_col="id", usecols=[0])
depths_df = pd.read_csv("../input/tgs-salt-identification-challenge/depths.csv", index_col="id")
train_df = train_df.join(depths_df)
test_df = depths_df[~depths_df.index.isin(train_df.index)]

len(train_df)
train_df["masks"] = [np.array(imread("./train/masks/{}.png".format(idx), as_gray=True)) / 255/257\
                     for idx in (train_df.index)]
img_size_ori = train_df["masks"][0].size
train_df["coverage"] = train_df.masks.map(np.sum) / img_size_ori
def cov_to_class(val):    
    for i in range(0, 11):
        if val * 10 <= i :
            return i
        
train_df["coverage_class"] = train_df.coverage.map(cov_to_class)

In [ ]:
ids_train, ids_valid =\
    train_test_split(
    train_df.index.values,
    test_size=0.2, stratify=train_df.coverage_class, random_state= 1234)

In [ ]:
x_imgs = [np.array(imread("./train/images/{}.png".format(idx), as_gray=True)) / 255\
                     for idx in ids_train]
y_imgs = [np.array(imread("./train/masks/{}.png".format(idx), as_gray=True)) / 255/257\
                     for idx in ids_train]

In [ ]:

x_train = np.concatenate((\
x_imgs, [np.fliplr(x) for x in x_imgs], [np.flipud(x) for x in x_imgs], [np.rot90(x) for x in x_imgs])
, axis = 0)

y_train = np.concatenate((\
y_imgs, [np.fliplr(x) for x in y_imgs], [np.flipud(x) for x in y_imgs], [np.rot90(x) for x in y_imgs])\
, axis = 0)

x_train = Tensor(x_train)
y_train = Tensor(y_train)

x_train = unsqueeze(x_train, 1)
y_train = unsqueeze(y_train, 1)

In [ ]:
x_val = [np.array(imread("./train/images/{}.png".format(idx), as_gray=True)) / 255\
                     for idx in ids_valid]
y_val = [np.array(imread("./train/masks/{}.png".format(idx), as_gray=True)) / 255/257\
                     for idx in ids_valid]
x_val = Tensor(x_val)
y_val = Tensor(y_val)

x_val = unsqueeze(x_val, 1)
y_val = unsqueeze(y_val, 1)

print(x_val.shape, y_val.shape)

In [ ]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)
    
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [ ]:

def batch_loader(x_train, y_train, batch_size):
    
    if not x_train.shape[0]%batch_size : 
        its = x_train.shape[0]//batch_size
        for batch in range(its):
            imgs = x_train[batch*batch_size: (batch+1)*batch_size]
            masks = y_train[batch*batch_size: (batch+1)*batch_size]
            yield imgs, masks, batch_size
    else: 
        its = x_train.shape[0]//batch_size
        for batch in range(its):
            imgs = x_train[batch*batch_size: (batch+1)*batch_size]
            masks = y_train[batch*batch_size: (batch+1)*batch_size]
            yield imgs, masks, batch_size
        
        imgs = x_train[its*batch_size: ]
        masks = y_train[its*batch_size:]
        yield imgs, masks, x_train.shape[0] - its*batch_size
    
SMOOTH = 1e-8
def iou_pytorch(outputs, labels):
    outputs = outputs>=0.5
    labels = labels==1
    outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    
    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    thresholded = torch.clamp(20 * (iou - 0.5), 0, 10) / 10  # This is equal to comparing with thresolds
    
    return thresholded

In [ ]:


def train(net, x_train, y_train, x_val, y_val,epochs,  batch_size, criterion,optimizer, scheduler = None, metrics = None):
    net.train()
    flag = True
    for epoch in (range(epochs)):

        total_loss = 0
        trained_imgs = 0
        for imgs, masks, cur_size in batch_loader(x_train, y_train, batch_size):
            masks_pred = net(imgs)
            loss = criterion(masks_pred, masks)*cur_size
            total_loss += float(loss)
            trained_imgs += cur_size
#             print("loss:",str.format('{0:.3f}', float(total_loss/ trained_imgs)), end = '\r')
            optimizer.zero_grad()
            if flag:
                loss.backward(retain_graph=True)
                flag = not flag
            else:
                loss.backward(retain_graph=False)
            optimizer.step()
#         print("End of Epoch", epoch+1, "with avg loss:", str.format('{0:.3f}', float(total_loss/x_train.shape[0])))
        
        total_val_loss = 0
        total_iou = 0
        for imgs, masks, cur_size in batch_loader(x_val, y_val, batch_size):
            masks_pred = net(imgs)
            val_loss = criterion(masks_pred, masks)*cur_size
            
            total_val_loss += float(val_loss)
            iou = float(torch.sum(metrics(masks_pred.squeeze(1), masks.squeeze(1))))

            total_iou += iou
#             print("loss:",str.format('{0:.3f}', float(val_loss/cur_size)), end = '\r')
        print("Epoch {}".format(epoch + 1))
        print("val_loss: ", str.format('{0:.3f}', float(total_val_loss/ x_val.shape[0])))
        print("val_iou: ", str.format('{0:.3f}', float(total_iou/ x_val.shape[0])))
        
        if scheduler != None and metrics != None:
            scheduler.step(total_iou)
            
net = UNet(1,1,  bilinear=False)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(net.parameters(), lr=0.001, weight_decay=1e-8, momentum=0.9)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max',factor = 0.1, patience=3, min_lr=1e-7)
epochs = 1
batch_size = 32

In [ ]:
train(net, x_train, y_train, x_val, y_val, 200, 64, criterion, optimizer, scheduler, iou_pytorch)

In [ ]:
torch.save(net.state_dict(), "./unet.pth")